In [1]:
import json
import replicate
import chromadb
from tqdm.auto import tqdm
import os

/packages/apps/mamba/1.5.1/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from replicate.client import Client

replicate = Client(api_token="r8_IbZu0U3qUJ5ZC0TzzMm6xCJ6yU5UxVi16Ejo4")

In [3]:
client = chromadb.PersistentClient(path="./persistent-client")

In [4]:
def add_text_files_to_collection(collection, datasets_dir):
  """
  Adds text files from a directory to a collection with generated IDs.

  Args:
      collection: The collection object to add documents and IDs to.
      datasets_dir: The directory containing the text files.
  """
  text_files = [os.path.join(datasets_dir, filename) for filename in os.listdir(datasets_dir)
                if filename.endswith(".txt")]

  if not text_files:
    print(f"No text files found in directory: {datasets_dir}")
    return

  start_id = 1 
  for i, filename in enumerate(text_files):
    document_id = f"doc_{start_id + i}"

    with open(filename, 'r') as f:
      document_content = f.read()
        
    collection.add(documents=[document_content], ids=[document_id])

In [5]:
collection = client.get_or_create_collection(
        name="aws_blogs",
        metadata={"hnsw:space": "cosine"}
    )
datasets_dir = "../dataset/aws-case-studies-blogs-dataset" 
add_text_files_to_collection(collection, datasets_dir)

2024-03-31 13:54:37.488415229 [E:onnxruntime:Default, env.cc:254 ThreadMain] pthread_setaffinity_np failed for thread: 3779348, index: 0, mask: {1, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2024-03-31 13:54:37.533519433 [E:onnxruntime:Default, env.cc:254 ThreadMain] pthread_setaffinity_np failed for thread: 3779349, index: 1, mask: {2, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2024-03-31 13:54:37.553714044 [E:onnxruntime:Default, env.cc:254 ThreadMain] pthread_setaffinity_np failed for thread: 3779350, index: 2, mask: {3, }, error code: 22 error msg: Invalid argument. Specify the number of threads explicitly so the affinity is not set.
2024-03-31 13:54:37.560185717 [E:onnxruntime:Default, env.cc:254 ThreadMain] pthread_setaffinity_np failed for thread: 3779351, index: 3, mask: {4, }, error code: 22 error msg: Invalid argument. Specify the n

In [6]:
query_string = "tell me about aws lambda"

results = collection.query(
    query_texts=[query_string],
    n_results=10,
)

In [7]:
successful_titles = ['\n'.join(results['documents'][0])]
successful_titles[:10]

['In addition to better compliance with regulations through improved transparency, the Purple IT team has improved and accelerated software development processes using AWS.\nAWS Lambda is a serverless, event-driven compute service that lets you run code for virtually any type of application or backend service without provisioning or managing servers. You can trigger Lambda from over 200 AWS services and software as a service (SaaS) applications, and only pay for what you use.File processing\xa0Stream processing\xa0Web applications\xa0IoT backends\xa0Mobile backends.  \nFrançais\n Benefits of AWS\nJan Červinka Director of Engineering, Purple Technology \nEspañol\n Amazon EC2\nThe Czech-based company builds apps that complement online trading platforms and support the changing and demanding needs of brokers. Purple’s solution enables tens of thousands of clients to trade many billions of dollars of assets each month.\n日本語\nIn addition, brokers and traders must comply with rules that chan

In [8]:
PROMPT_TEMPLATE = f'''[INST]
You are an expert in all things hackernews. Your goal is to help me write the most click worthy hackernews title that will get the most upvotes. You will be given a USER_PROMPT, and a series of SUCCESSFUL_TITLES. You will respond with 5 suggestions for better hackernews titles.

All of your suggestions should be structured in the same format and tone as the previously successful SUCCESSFUL_TITLES. Make sure you do not include specific versions from the SUCCESSFUL_TITLES in your suggestions.

USER_PROMPT: {query_string}

SUCCESSFUL_TITLES: {successful_titles}

SUGGESTIONS:

[/INST]
'''

mistral_response = replicate.run(
    "a16z-infra/mistral-7b-instruct-v0.1:83b6a56e7c828e667f21fd596c338fd4f0039b46bcfa18d973e8e70e455fda70",
    input={
        "prompt": PROMPT_TEMPLATE,
        "temperature": 0.75,
        'max_new_tokens': 2048,
    },
)

suggestions = ''.join([str(s) for s in mistral_response])

print(suggestions)

print('====')

print('PROMPT_TEMPLATE', PROMPT_TEMPLATE)

ModelError: 